In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNet
import pandas as pd

In [2]:
data1 = pd.read_csv("corvCleanDataset.csv")
data1.head()

#making a working copy
data = data1.copy()

In [3]:
keys = list(data.keys())
keys.remove('Date')
keys.remove('Open')
keys.remove('High')
keys.remove('Low')
keys.remove('Close')
keys.remove('Volume')
keys.remove('Ticker')

In [4]:
xes = keys
ys = "High"
print(xes)
train, test = train_test_split(data, test_size=0.2, random_state=42)
x_train = train[keys]
y_train = train[ys]
x_test = test[keys]
y_test = test[ys]

['high1DayBefore', 'low1DayBefore', 'open1DayBefore', 'close1DayBefore', 'volume1DayBefore', 'high2DaysBefore', 'low2DaysBefore', 'open2DaysBefore', 'close2DaysBefore', 'volume2DaysBefore', 'high3DaysBefore', 'low3DaysBefore', 'open3DaysBefore', 'close3DaysBefore', 'volume3DaysBefore', 'high4DaysBefore', 'low4DaysBefore', 'open4DaysBefore', 'close4DaysBefore', 'volume4DaysBefore', 'high5DaysBefore', 'low5DaysBefore', 'open5DaysBefore', 'close5DaysBefore', 'volume5DaysBefore']


In [5]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

looking at the scores for the training set.

In [6]:
print(reg.coef_)
print("intercept is " + str(reg.intercept_))
r2Train = reg.score(x_train,y_train)
y_train_predicted = reg.predict(x_train)
train_ms_error = mean_squared_error(y_train, y_train_predicted)
print("the mean squared error is " + str(train_ms_error))
print("r2 on the train set is " + str(r2Train))

[ 1.36669237e-01 -9.13318479e-02  2.80847247e-02  9.49168162e-01
  8.69736230e-09  9.60897809e-02 -2.51944568e-02 -3.38241313e-02
 -5.37974079e-02  6.68594881e-09 -3.72473352e-02 -1.62619521e-01
  9.12440147e-02  1.35610514e-01  1.60812940e-09 -4.33567855e-02
  3.83924337e-04  5.47123943e-02 -1.80234943e-02 -1.18771370e-08
  1.22912646e-03 -1.06896139e-01  7.36415087e-02  1.41508730e-02
 -1.34755570e-08]
intercept is 0.04734706389240273
the mean squared error is 0.8932355413448684
r2 on the train set is 0.9973378381556776


### Did really good on my set overall. now i should test it on the test sets. Then i will get a more representative score.

In [7]:
r2Test = reg.score(x_test,y_test)
y_test_predicted = reg.predict(x_test)
test_ms_error = mean_squared_error(y_test, y_test_predicted)
print("the mean squared error is " + str(test_ms_error))
print("r2 on the train set is " + str(r2Test))

the mean squared error is 0.6205378485915528
r2 on the train set is 0.9981701225951763


In [14]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
scores = cross_val_score(tree_reg,data[keys],data[ys],scoring="neg_mean_squared_error",cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [15]:
print(tree_rmse_scores)

[2.00768708 2.72133384 3.24556562 2.13302383 1.74658416 0.98893798
 0.3452369  0.50240784 0.19326024 0.5774038 ]


grid search 

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [{'n_estimators': [50,60,80,100,200], 'max_features': [8,10,11,12,13,14,15,20]},
             {'bootstrap': [False], 'n_estimators': [50,60,80,100,200], 'max_features': [8,10,11,12,13,14,15,20]},]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid,cv=5,scoring='neg_mean_squared_error')
grid_search.fit(data[keys],data[ys])
best = grid_search.best_estimator_
best.score(test[keys],test[ys])

0.9996811536021173

In [22]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 50}

In [23]:
best = grid_search.best_estimator_

In [24]:
best.score(test[keys],test[ys])

0.9996392239997798